In [1]:
import numpy as np
import pandas as pd
import json
import glob
import re
from tqdm import tqdm
import time

In [2]:
disclosures = glob.glob('./DATA/DISCLOSURES/*.txt')
corplist = pd.read_csv('./DATA/Corplist.csv')

In [3]:
def getBody(doc):
    for element in doc:
        if 'II. 사업의 내용' in element:
            page1 = str(int(element.strip().split('.')[-1])-1)
        if 'III. 재무에 관한 사항' in element:
            page2 = str(int(element.strip().split('.')[-1])-1)
            break

    for element in doc:
        if 'Page '+page1+'\n' in element:
            index1 = doc.index(element)
        if 'Page '+str(int(page2)+1)+'\n' in element:
            index2 = doc.index(element)
            break
    body = doc[index1+1:index2]

    for index, line in enumerate(reversed(body)):
        if 'III. 재무에 관한 사항' in line:
            break
    
    if index == len(body)-1:
        return body        
    
    else:
        index2 = len(body) - index
        body = body[:index2-1]

        for index, line in enumerate(body):
            if 'II. 사업의 내용' in line:
                break
        body = body[index:]

        return body

In [4]:
def clearing(body):
    lenlist = [len(element) for element in body]
    delidx = []
    i=0
    start = 0
    end = 0
    while(i<len(body)):
        if  lenlist[i]==1:
            start = i
            end = i
            while(lenlist[end]==1):
                end+=1
                if end==len(body):
                    break
            i = end
            if (end-start)>1:
                delidx.append((start,end))
        else:
            i+=1
    delidx.reverse()

    for element in delidx:
        del body[element[0]:element[1]-1]
    
    return body

In [5]:
def deleteTable(body):
    preserve = ['II.','1. ', '2. ', '3. ', '4. ', '5. ', '6. ', '7. ', '8. ', '9. ', '(1)', '(2)', '(3)', '(4)', '(5)',
            '(6)', '(7)', '(8)', '(9)']
    delete = []
    for i in range(len(body)-2):
        if body[i].strip()=='':
            if body[i+2].strip()=='':
                if body[i+1][:3] not in preserve:
                    delete.append(i+1)
    delete.reverse()
    for index in delete:
        del body[index]
    body = clearing(body)
    
    return body

In [6]:
def deleteMeaningless(body):
    delete = []
    for i in range(len(body)-2):
        if body[i].strip()=='':
            if body[i+2].strip()=='':
                delete.append(i+1)
    delete.reverse()
    for index in delete:
        del body[index]
    body = clearing(body)
    
    erase = []
    for i in range(len(body)):
        if len(body[i].split(' '))<=2:
            erase.append(i)
    erase.reverse()
    for index in erase:
        del body[index]
    
    return ' '.join(body)

#### 'II. 사업의 내용' 텍스트 추출

In [8]:
for index, disclosure in tqdm(enumerate(disclosures)):
    with open(disclosure) as f:
        doc = f.readlines()
        body = getBody(doc)
        #body = clearing(body)
        body = deleteTable(body)
        body = '\n'.join(body)
        #body = deleteMeaningless(body)
        body = body.replace('\x0c','')
        body = body.replace('\n ','\n')
        body = body.replace('\n\n ','\n')
        body = body.replace('\n\n','\n')
        body = body.replace('\n\n\n ','\n')
        body = body.replace('\n\n\n','\n')
    with open(disclosure.replace('DISCLOSURES','DISCLOSURES_preprocessed'), 'w') as f:
        f.write(body)

760it [00:17, 43.64it/s]


In [108]:
df = pd.DataFrame(columns=['Code','Name','Business Section'])
for index, disclosure in tqdm(enumerate(disclosures)):
    with open(disclosure) as f:
        doc = f.readlines()
        body = getBody(doc)
        body = clearing(body)
        body = deleteTable(body)
        body = deleteMeaningless(body)
        body = body.replace('\x0c','')
        body = body.replace('\n ',' ')
        body = body.replace('\n',' ')
        body = body.replace('II. 사업의 내용 ','')
    code = 'A'+disclosure.split('_')[-1].split('.')[0]
    name = corplist[corplist.Code == code].iloc[0].Name
    df.loc[index] = [code, name, body]

760it [00:12, 63.03it/s]


In [112]:
df.to_csv(r'./DATA/extracted_business_section.csv', index=False, encoding='utf-8')